# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoModelForCausalLM
from datasets import load_dataset
import numpy as np
import evaluate
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM

- Prepare a dataset

In [12]:
dataset = load_dataset("ccdv/arxiv-classification")

- Tokenizer to process the text include a padding and truncation strategy to handle any variable sequence lengths. Using Hugging Face's datasets map method to apply a preprocessing function over the entire dataset:

In [13]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 2500/2500 [00:33<00:00, 74.35 examples/s]


- Create a smaller subset of the full dataset to fine-tune on to reduce the time it takes

In [14]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

- Loading model and specify the number of expected labels.

In [15]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=11)

model.safetensors: 100%|██████████| 436M/436M [00:07<00:00, 57.9MB/s] 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


- Create a TrainingArguments class which contains all the hyperparameters we can tune as well as flags for activating different training options then specify where to save the checkpoints from training

In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

- Evaluate

In [18]:
metric = evaluate.load("accuracy")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

- Create a Trainer object

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

- Fine-tune your model by calling train():

In [21]:
trainer.train()

100%|██████████| 375/375 [04:41<00:00,  1.33it/s]

{'train_runtime': 281.7849, 'train_samples_per_second': 10.646, 'train_steps_per_second': 1.331, 'train_loss': 1.083978759765625, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=1.083978759765625, metrics={'train_runtime': 281.7849, 'train_samples_per_second': 10.646, 'train_steps_per_second': 1.331, 'train_loss': 1.083978759765625, 'epoch': 3.0})

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

- Creating a PEFT Config

In [22]:
config = LoraConfig()

- Converting a Transformers Model into a PEFT Model

In [23]:
model = AutoModelForCausalLM.from_pretrained("gpt2")

config.json: 100%|██████████| 665/665 [00:00<00:00, 44.6kB/s]
model.safetensors: 100%|██████████| 548M/548M [00:07<00:00, 69.5MB/s] 
generation_config.json: 100%|██████████| 124/124 [00:00<00:00, 122kB/s]


- Training with a PEFT Model And Checking Trainable Parameters of a PEFT Model

In [24]:
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.23643136409814364


/Users/hsin-wenchang/Documents/GitHub/Apply-Lightweight-Fine-Tuning-to-a-Foundation-Model/.env/lib/python3.9/site-packages/peft/tuners/lora/layer.py:711: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


- Saving a Trained PEFT Model

In [25]:
lora_model.save_pretrained("gpt-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

- Loading a Saved PEFT Model

In [26]:
lora_model = AutoPeftModelForCausalLM.from_pretrained("gpt-lora")

- Generating Text from a PEFT Model

In [27]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
inputs = tokenizer("Hello, my name is ", return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
print(tokenizer.batch_decode(outputs))

vocab.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 12.2MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 18.7MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 59.1MB/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Hello, my name is _____. I am a student at the University of']
